<a href="https://colab.research.google.com/gist/chezou/8cc7e5a3eedd795a56a85204d8ca10ef/ex2-3_plot_saled_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 演習 2-3 売上データの予測結果をグラフに描画しよう

Facebook Prophetで予測をした売上をJupyter notebookを使いプロットしてみましょう。

## pytdをインストールする

このセルはローカル環境では実行しなくても構いませんが、Google Colaboratoryなどの環境では初回実行時に必ず実行してください。

In [0]:
# Install pytd
!pip install -q pytd

## TD API keyを設定する

TDの読み書きができるmaster API keyを設定します。

API keyは以下のURLから確認できます。
https://console.treasuredata.com/app/mp/ak

"Input your Treasure Data API key"の下のボックスにAPIキーをコピー＆ペーストをしてください。

In [0]:
# Set your TD API key
import getpass

print("Input your Treasure Data API key")
apikey = getpass.getpass()
print("Succeeded")

## 各種設定を変更する

下のセルで `db_name` の値を `timeseries` から `timeseries_YOURNAME` のようにYOURNAMEの部分を名前などに置き換えてください。
また、USリージョン以外の場合は、 `endpoint` も適宜書き換えてください。

- 東京: "https://api.treasuredata.co.jp"
- EU: "https://api.eu01.treasuredata.com"

In [0]:
# Set configurations
db_name = "timeseries"
endpoint = "https://api.treasuredata.com"

## グラフ描画用の関数を定義する

次のセルではProphetのグラフを描画するための関数を定義しています。詳細は飛ばしてしまって構いません。

In [0]:
%matplotlib inline

# Define function for plot

from matplotlib import pyplot as plt
from matplotlib.dates import (
    AutoDateLocator,
    AutoDateFormatter,
)

# Equivalent code with model.plot()
# https://github.com/facebook/prophet/blob/ca9a49d328ab1f2a991f246a3ebc37a7f9c896c5/python/fbprophet/plot.py#L41-L88
def plot(df, df_prev, xlabel='ds', ylabel='y'):
    fig = plt.figure(facecolor="w", figsize=(10,6))
    ax = fig.add_subplot(111)

    fcst_t = pd.to_datetime(df["ds"])
    ax.plot(pd.to_datetime(df_prev["ds"]), df_prev["y"], "k.")
    ax.plot(fcst_t, df["yhat"], ls="-", c="#0072B2")
    if "cap" in df:
        ax.plot(fcst_t, df["cap"], ls="--", c="k")

    if "floor" in df:
        ax.plot(fcst_t, df["floor"], ls="--", c="k")

    ax.fill_between(fcst_t, df["yhat_lower"], df["yhat_upper"], color="#0072B2", alpha=0.2)
    locator = AutoDateLocator(interval_multiples=False)
    formatter = AutoDateFormatter(locator)
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(formatter)
    ax.grid(True, which='major', c='gray', ls='-', lw=1, alpha=0.2)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    fig.tight_layout()

    return fig

## TDから過去の売上データを取得する

過去の売上データをpytdを使い取得します。また、取得しデータを折れ線グラフでプロットします。

In [0]:
# Load past sales data
import pytd
import pandas as pd

pytd_client = pytd.Client(database=db_name, apikey=apikey, endpoint=endpoint)
df_prev = pd.DataFrame(**pytd_client.query("select * from retail_sales order by ds"))
df_prev

In [0]:
# Plot past sales data
df_prev.plot(x="ds", y="y", kind="line", figsize=(10, 6))

## 予測結果をTDから取得する

Custom Scripts環境で予測した結果をpytdで取得し、その結果をプロットします。

In [0]:
# Load predicted sales data
df = pd.DataFrame(**pytd_client.query("select * from predicted_sales order by ds"))
df

In [0]:
# Plot predicted sales data
fig = plot(df, df_prev)